In [1]:
# Add base directory for forestfire project to path to allow use of utils
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import cv2
from skimage.metrics import structural_similarity
from os import listdir
from os.path import isdir
import numpy as np
import PIL
import numpy as np
import cv2
import matplotlib.pyplot as plt

from IPython.display import Image
from IPython.display import display

import seaborn as sns
import gc
from multiprocessing import Pool
from utils.image_utils import rmse, ssim_score, video_iterator, blur_score

In [3]:
def list_files_with_extension(directory, extension):
    return (f for f in listdir(directory) if f.endswith('.' + extension))

In [4]:
def generate_significant_frames(input_path, input_file, output_image_path, compare_function, threshold,
                                downsample_size, greyscale, frames_per_second, show_frames=False):
    
    vid_frame_list = list(video_iterator(input_path, input_file, frames_per_second))
    
    if vid_frame_list is None:
        return []
    
    sig_frames = []
    last_sig_frame = None
    for i in range(len(vid_frame_list) - 1):
        curr_frame = vid_frame_list[i + 1]
        
        if last_sig_frame is None:
            _blur_score = blur_score(curr_frame, downsample_size)
            if _blur_score >= 75:
                last_sig_frame = curr_frame
                sig_frames += [(curr_frame, input_file + "_" + str(i))]

        else:
            score = compare_function(last_sig_frame, curr_frame, downsample_size=downsample_size, greyscale=greyscale)
            if score <= threshold:
                if blur_score(curr_frame, downsample_size) >= 200:
                    last_sig_frame = curr_frame
                    sig_frames += [(curr_frame, input_file + "_" + str(i))]
    
    for img in sig_frames:
        if show_frames:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.show()
        cv2.imwrite(output_image_path + "/" + str(img[1])  + ".jpg", img[0])
    
    print("Frame generation for", input_file, "complete - ", len(vid_frame_list), "frames reduced to", len(sig_frames))

In [33]:
# Change this to suit your environment
base_path = "/notebooks/storage/forestfire/data"
input_path = base_path + "/new_videos/"
output_image_path = base_path + "/videos/frame_extracts_new_periodic/"
video_zip_path = "https://sintecsys-omdena.s3.amazonaws.com/new_videos/"
video_zip_name = "new_videos.zip"

In [34]:
# if isdir(input_path) == False:
#     !mkdir $input_path
# if isdir(output_image_path) == False:
#     !mkdir $output_image_path
# !cd {input_path} \
#     && curl -O {video_zip_path + video_zip_name}

In [35]:
# !cd {input_path} && \
#     sudo unzip -o -q {video_zip_name} && \
#     sudo chmod -R u+rw  ./ && \
#     sudo mv -f {"./" + video_zip_name.split(".")[0] + "/*.mp4"} ./ && \
#     sudo rm -Rf {"./" +video_zip_name.split(".")[0]} ./__*

In [36]:
video_list = sorted(list(list_files_with_extension(input_path, "mp4")) + list(list_files_with_extension(input_path, "dav")))

In [37]:
video_list

['20191104140000_20191104143000_100_F_Main.dav',
 '20191104140000_20191104143000_200_F_Main.dav',
 '20191104140000_20191104143000_400_F_Main.dav',
 '20191104140000_20191104143000_800_F_Main.dav',
 '20191104140000_20191104144000_1000_F_Main.dav',
 '20191104140000_20191104144000_800_F_Main.dav',
 'T1_15h.mp4',
 'T3_15h30.mp4',
 'T4_15h47.mp4']

In [38]:
# def gsf_for_pool(video):
#     return generate_significant_frames(input_path, str(video), output_image_path, compare_function=ssim_score,
#                                                       threshold=0.275, downsample_size=(1080//4, 1920//4),
#                                                       greyscale=True, frames_per_second=2, show_frames=False)

In [39]:
def gsf_for_pool(video):
    return generate_significant_frames(input_path, str(video), output_image_path, compare_function=ssim_score,
                                                      threshold=1, downsample_size=(1080//4, 1920//4),
                                                      greyscale=True, frames_per_second=1, show_frames=False)

In [40]:
if __name__ == '__main__':
    p = Pool(8)
    p.map(gsf_for_pool, video_list)

Frame generation for 20191104140000_20191104143000_200_F_Main.dav complete -  0 frames reduced to 0
Frame generation for 20191104140000_20191104143000_400_F_Main.dav complete -  1 frames reduced to 0
Frame generation for 20191104140000_20191104143000_800_F_Main.dav complete -  1 frames reduced to 0
Frame generation for T1_15h.mp4 complete -  302 frames reduced to 301
Frame generation for T4_15h47.mp4 complete -  302 frames reduced to 301
Frame generation for T3_15h30.mp4 complete -  302 frames reduced to 301
Frame generation for 20191104140000_20191104143000_100_F_Main.dav complete -  1798 frames reduced to 1797
Frame generation for 20191104140000_20191104144000_1000_F_Main.dav complete -  600 frames reduced to 597
Frame generation for 20191104140000_20191104144000_800_F_Main.dav complete -  2398 frames reduced to 2287
